<h1><center>PI #1 MLOPS - ETL</center></h1>

### Importando librerias

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


### Transformando el archivo "movies_dataset"

In [2]:
# Carga de archivos
df = pd.read_csv("Dataset/movies_dataset.csv") 
df.head(3)

C:\Users\ferca\AppData\Local\Temp\ipykernel_9860\3842962304.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Dataset/movies_dataset.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [3]:
# breve analisis de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

### Características de los datos:	Descripción
- **adult:**	Indica si la película tiene califiación X, exclusiva para adultos.(**categorica: True o False**)
- **belongs_to_collection:**	Un diccionario que indica a que franquicia o serie de películas pertenece la película (**categorica**)
- **budget:**	El presupuesto de la película, en dólares (**numerica**)
- **genres:**	Un diccionario que indica todos los géneros asociados a la película (**categorica**)
- **homepage:**	La página web oficial de la película (**categorica**)
- **id:**	ID de la pelicula (**nominal**)
- **imdb_id:**	IMDB ID de la pelicula (**nominal**)
- **original_language:**	Idioma original en la que se grabo la pelicula (**categorica**)
- **original_title:**	Titulo original de la pelicula (**categorica**)
- **overview:**	Pequeño resumen de la película (**categorica**)
- **popularity:**	Puntaje de popularidad de la película, asignado por TMDB (TheMoviesDataBase) (**numerico**)
- **poster_path:**	URL del póster de la película (**categorica**)
- **production_companies:**	Lista con las compañias productoras asociadas a la película (**categorica**)
- **production_countries:**	Lista con los países donde se produjo la película (**categorica**)
- **release_date:**	Fecha de estreno de la película (**numerica**)
- **revenue:**	Recaudación de la pelicula, en dolares (**numerica**)
- **runtime:**	Duración de la película, en minutos (**numerica**)
- **spoken_languages:**	Lista con los idiomas que se hablan en la pelicula (**categorica**)
- **status:**	Estado de la pelicula actual (si fue anunciada, si ya se estreno, etc) (**categorica**)
- **tagline:**	Frase celebre asociadaa la pelicula (**categorica**)
- **title:**	Titulo de la pelicula (**categorica**)
- **video:**	Indica si hay o no un trailer en video disponible en TMDB (**categorica: True o False**)
- **vote_average:**	Puntaje promedio de reseñas de la pelicula (**numerica**)
- **vote_count:**	Numeros de votos recibidos por la pelicula, en TMDB (**numerica**)


### 1) Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [4]:
# analizando columnas innecesarias
valores_unicos = df['adult'].value_counts()
print(valores_unicos)

adult
False                                                                                                                             45454
True                                                                                                                                  9
 - Written by Ørnås                                                                                                                   1
 Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.                        1
 Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.        1
Name: count, dtype: int64


In [5]:
# eliminando 3 filas innecesarias, se opto por eliminar porque solo hay tres filas que no coinciden con el tipo de dato
df = df[df['adult'].isin(["True", "False"])]

In [6]:
#validando borrado de filas
valores_unicos = df['adult'].value_counts()
print(valores_unicos)

adult
False    45454
True         9
Name: count, dtype: int64


In [7]:
# Lista de columnas a eliminar
columns_a_eliminar = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']

# Eliminar las columnas no utilizadas
df.drop(columns=columns_a_eliminar, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45463 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget                 45463 non-null  object 
 2   genres                 45463 non-null  object 
 3   id                     45463 non-null  object 
 4   original_language      45452 non-null  object 
 5   overview               44509 non-null  object 
 6   popularity             45460 non-null  object 
 7   production_companies   45460 non-null  object 
 8   production_countries   45460 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20412 non-null  object 
 15  title  

### 2) Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [8]:
# Verificar y convertir los valores no nulos en la columna 'belongs_to_collection'
df['belongs_to_collection'] = df['belongs_to_collection'].apply(lambda x: ast.literal_eval(x)['name'] if not pd.isnull(x) else np.nan)

# Función para obtener los names de las columnas
def obtener_names(columna_str):
    try:
        name_list = ast.literal_eval(columna_str)
        return ', '.join([name['name'] for name in name_list])
    except:
        return ''

# Aplicar la función a la columna "genres", "production_companies", "production_countries" y "spoken_languages"
df['genres'] = df['genres'].apply(obtener_names)
df["production_companies"] = df["production_companies"].apply(obtener_names)
df["production_countries"] = df["production_countries"].apply(obtener_names)
df["spoken_languages"] = df["spoken_languages"].apply(obtener_names)

df.head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,Toy Story Collection,30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0
1,NaN,65000000,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,1995-12-15,262797249.0,104.0,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,Grumpy Old Men Collection,0,"Romance, Comedy",15602,en,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros., Lancaster Gate",United States of America,1995-12-22,0.0,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0


### 3) Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0

In [9]:
# Función contar valores nulos
def contar_valores_nulos(columna):
    num_nulos = df[columna].isnull().sum()
    print("Número de valores nulos en la columna", columna , ":", num_nulos)

# LLamando a la función
contar_valores_nulos("budget")
contar_valores_nulos("revenue")

Número de valores nulos en la columna budget : 0
Número de valores nulos en la columna revenue : 3


In [10]:
# Reemplazando valores nulos por cero y validando
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

contar_valores_nulos("budget")
contar_valores_nulos("revenue")

Número de valores nulos en la columna budget : 0
Número de valores nulos en la columna revenue : 0


### 4) Los valores nulos del campo release date deben eliminarse.

In [11]:
# Contar los valores nulos en la columna "release_date"
contar_valores_nulos("release_date")

# Eliminar los valores nulos en la columna 'release_date'
df.dropna(subset=['release_date'], inplace=True)

# Contar los valores nulos en la columna "release_date"
contar_valores_nulos("release_date")

Número de valores nulos en la columna release_date : 87
Número de valores nulos en la columna release_date : 0


### 5) De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [12]:
# Estableciendo formato de fecha y creando columna release_year
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_year'] = df['release_date'].dt.year
df[["release_date", "release_year"]].head(3)

,release_date,release_year
0,1995-10-30,1995
1,1995-12-15,1995
2,1995-12-22,1995


### 6) Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [13]:
# creando la columna return
df["revenue"] = df["revenue"].replace(0, np.nan)
df["budget"] = pd.to_numeric(df["budget"], errors ="coerce")
df["budget"] = df["budget"].replace(0, np.nan)

df["return"] = df["revenue"] / df["budget"]
print(df[df["return"].isnull()].shape)
print(df[df["budget"].isnull()].shape)

(39995, 20)
(36490, 20)


In [14]:
# cambiando formato a la columna "id" para luego hacer un merge con el archivo "credits"
df['id'] = pd.to_numeric(df['id'])

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4488 non-null   object        
 1   budget                 8886 non-null   float64       
 2   genres                 45376 non-null  object        
 3   id                     45376 non-null  int64         
 4   original_language      45365 non-null  object        
 5   overview               44435 non-null  object        
 6   popularity             45376 non-null  object        
 7   production_companies   45376 non-null  object        
 8   production_countries   45376 non-null  object        
 9   release_date           45376 non-null  datetime64[ns]
 10  revenue                7407 non-null   float64       
 11  runtime                45130 non-null  float64       
 12  spoken_languages       45376 non-null  object        
 13  status

In [15]:
df.head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
0,Toy Story Collection,30000000.0,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0,1995,12.451801
1,NaN,65000000.0,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,1995-12-15,262797249.0,104.0,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035
2,Grumpy Old Men Collection,NaN,"Romance, Comedy",15602,en,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros., Lancaster Gate",United States of America,1995-12-22,NaN,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,NaN


### Transformando el archivo "credits"

In [16]:
#Leemos el archivo csv
df2 = pd.read_csv("Dataset/credits.csv") 
df2.head(3)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


In [17]:
# Función para extraer los nombres de los directores
def extraer_director_names(row):
    crew_list = ast.literal_eval(row['crew'])  # Convertir la cadena a una lista de diccionarios
    directors = [crew['name'] for crew in crew_list if crew['job'] == 'Director']
    return ', '.join(directors)

# Aplicar la función a la columna 'crew' y asignar el resultado a una nueva columna 'directors'
df2['crew'] = df2.apply(extraer_director_names, axis=1)

# Aplicar la función a la columna "cast"
df2['cast'] = df2['cast'].apply(obtener_names)

df2.head(3)

,cast,crew,id
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter,862
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston,8844
2,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch,15602


### Uniendo el dataset "movies_dataset" y "credits" atraves de la columna "id"

In [18]:
# Combinar los DataFrames df1 y df2 utilizando la columna 'id'
dff = df.merge(df2[['id', 'cast', 'crew']], on='id', how='left')

df.head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
0,Toy Story Collection,30000000.0,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0,1995,12.451801
1,NaN,65000000.0,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,1995-12-15,262797249.0,104.0,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035
2,Grumpy Old Men Collection,NaN,"Romance, Comedy",15602,en,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros., Lancaster Gate",United States of America,1995-12-22,NaN,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,NaN


In [19]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45452 entries, 0 to 45451
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4497 non-null   object        
 1   budget                 8908 non-null   float64       
 2   genres                 45452 non-null  object        
 3   id                     45452 non-null  int64         
 4   original_language      45441 non-null  object        
 5   overview               44511 non-null  object        
 6   popularity             45452 non-null  object        
 7   production_companies   45452 non-null  object        
 8   production_countries   45452 non-null  object        
 9   release_date           45452 non-null  datetime64[ns]
 10  revenue                7427 non-null   float64       
 11  runtime                45206 non-null  float64       
 12  spoken_languages       45452 non-null  object        
 13  s

In [20]:
dff.to_csv("Dataset_generados/movies_dataset_transformado.csv", index = False)

df = pd.read_csv("Dataset_generados/movies_dataset_transformado.csv")
df. head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,cast,crew
0,Toy Story Collection,30000000.0,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,...,English,Released,NaN,Toy Story,7.7,5415.0,1995,12.451801,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,NaN,65000000.0,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,1995-12-15,...,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,Grumpy Old Men Collection,NaN,"Romance, Comedy",15602,en,A family wedding reignites the ancient feud be...,11.712900,"Warner Bros., Lancaster Gate",United States of America,1995-12-22,...,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,NaN,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch


### Exportando archivos independientes para el uso de las funciones

#### Exportando el dataset para la funcion: peliculas_idioma( Idioma: str )

In [162]:
idiomas = df.groupby('original_language').size().reset_index(name='cantidad_peliculas')
idiomas.head(89)

,original_language,cantidad_peliculas
0,ab,10
1,af,2
2,am,2
3,ar,39
4,ay,1
...,...,...
84,vi,10
85,wo,5
86,xx,33
87,zh,408


In [163]:
idiomas.to_csv("Dataset_generados/idiomas.csv", index = False)

df = pd.read_csv("Dataset_generados/idiomas.csv")
df. head()

,original_language,cantidad_peliculas
0,ab,10
1,af,2
2,am,2
3,ar,39
4,ay,1


#### Exportando el dataset para la funcion: peliculas_duracion( Pelicula: str )

In [164]:
duracion = dff[['title', 'runtime', 'release_year']]
duracion.head()


,title,runtime,release_year
0,Toy Story,81.0,1995
1,Jumanji,104.0,1995
2,Grumpier Old Men,101.0,1995
3,Waiting to Exhale,127.0,1995
4,Father of the Bride Part II,106.0,1995


In [165]:
duracion.to_csv("Dataset_generados/duracion.csv", index = False)

df = pd.read_csv("Dataset_generados/duracion.csv")
df. head()

,title,runtime,release_year
0,Toy Story,81.0,1995
1,Jumanji,104.0,1995
2,Grumpier Old Men,101.0,1995
3,Waiting to Exhale,127.0,1995
4,Father of the Bride Part II,106.0,1995


#### Exportando el dataset para la funcion: franquicia( Franquicia: str )

In [166]:
franquicia = dff.dropna(subset=['revenue']).groupby('belongs_to_collection').agg({'belongs_to_collection': 'count', 'revenue': ['sum', 'mean']})
franquicia.columns = ['cantidad_peliculas', 'ganancia_total', 'promedio_ganancia']
franquicia.reset_index(inplace=True)
franquicia.head()

,belongs_to_collection,cantidad_peliculas,ganancia_total,promedio_ganancia
0,... Has Fallen Collection,2,366780087.0,183390043.5
1,101 Dalmatians (Animated) Collection,1,215880014.0,215880014.0
2,101 Dalmatians (Live-Action) Collection,2,504301065.0,252150532.5
3,12 Rounds Collection,1,17280326.0,17280326.0
4,2 Days In... Collection,2,21429951.0,10714975.5


In [167]:
franquicia.to_csv("Dataset_generados/franquicia.csv", index = False)

df = pd.read_csv("Dataset_generados/franquicia.csv")
df. head()

,belongs_to_collection,cantidad_peliculas,ganancia_total,promedio_ganancia
0,... Has Fallen Collection,2,366780087.0,183390043.5
1,101 Dalmatians (Animated) Collection,1,215880014.0,215880014.0
2,101 Dalmatians (Live-Action) Collection,2,504301065.0,252150532.5
3,12 Rounds Collection,1,17280326.0,17280326.0
4,2 Days In... Collection,2,21429951.0,10714975.5


#### Exportando el dataset para la funcion: peliculas_pais( Pais: str )

In [169]:
pais = dff['production_countries'].str.split(',').explode().str.strip().value_counts().reset_index()
pais.columns = ['pais', 'cantidad_peliculas']
pais.head()

,pais,cantidad_peliculas
0,United States of America,21176
1,,6220
2,United Kingdom,4100
3,France,3956
4,Germany,2266


In [170]:
pais.to_csv("Dataset_generados/pais.csv", index = False)

df = pd.read_csv("Dataset_generados/pais.csv")
df.head()

,pais,cantidad_peliculas
0,United States of America,21176
1,NaN,6220
2,United Kingdom,4100
3,France,3956
4,Germany,2266


#### Exportando el dataset para la funcion: productoras_exitosas( Productora: str )

In [152]:
productoras = dff['production_companies'].str.split(', ').explode().str.strip().to_frame()
productoras['revenue'] = dff['revenue'] / dff['production_companies'].str.split(', ').apply(len)
productoras['cantidad_peliculas'] = dff['production_companies'].str.split(', ').apply(len)
productoras = productoras.groupby('production_companies').agg({'revenue': 'sum', 'cantidad_peliculas': 'sum'}).reset_index()
productoras = productoras.loc[(productoras['revenue'] != 0) & productoras['revenue'].notnull()]
productoras.columns = ['production_companies', 'revenue', 'cantidad_peliculas']

productoras.head()

,production_companies,revenue,cantidad_peliculas
0,,1.759118e+09,11826
1,"""DIA"" Productions GmbH & Co. KG",1.108773e+07,4
10,1.85 Films,1.225212e+04,9
12,100 Bares,6.440649e+06,18
17,1000 Volt,4.624333e+04,3


In [176]:
productoras.to_csv("Dataset_generados/productoras.csv", index = False)

df = pd.read_csv("Dataset_generados/productoras.csv")
df.head()

,production_companies,revenue,cantidad_peliculas
0,NaN,1.759118e+09,11826
1,"""DIA"" Productions GmbH & Co. KG",1.108773e+07,4
2,1.85 Films,1.225212e+04,9
3,100 Bares,6.440649e+06,18
4,1000 Volt,4.624333e+04,3


#### Exportando el dataset para la funcion: get_director( nombre_director )

In [89]:
# Exportamos un primer dataset que contiene solo el nombre del director y el retorno total de este
director_1 = dff.groupby('crew')['return'].sum().reset_index()
director_1 = director_1.loc[(director_1['return'] != 0) & director_1['return'].notnull()]
director_1.columns = ['director', 'return_total']
director_1.head()


,director,return_total
0,,4.293455
5,Yeon Sang-Ho,0.241470
17,A.R. Murugadoss,15.532898
23,Aamir Khan,10.934615
36,Aaron Katz,6.170000


In [91]:
director_1.to_csv("Dataset_generados/director_1.csv", index = False)

df = pd.read_csv("Dataset_generados/director_1.csv")
df.head()

,director,return_total
0,NaN,4.293455
1,Yeon Sang-Ho,0.241470
2,A.R. Murugadoss,15.532898
3,Aamir Khan,10.934615
4,Aaron Katz,6.170000


In [94]:
# Exportamos un segundo dataset que contiene los demas datos
director_2 = dff[["crew", "title", "budget", "revenue", "return", "release_year"]]
director_2.head()

,crew,title,budget,revenue,return,release_year
0,John Lasseter,Toy Story,30000000.0,373554033.0,12.451801,1995
1,Joe Johnston,Jumanji,65000000.0,262797249.0,4.043035,1995
2,Howard Deutch,Grumpier Old Men,NaN,NaN,NaN,1995
3,Forest Whitaker,Waiting to Exhale,16000000.0,81452156.0,5.090760,1995
4,Charles Shyer,Father of the Bride Part II,NaN,76578911.0,NaN,1995


In [95]:
director_2.to_csv("Dataset_generados/director_2.csv", index = False)

df = pd.read_csv("Dataset_generados/director_2.csv")
df.head()

,crew,title,budget,revenue,return,release_year
0,John Lasseter,Toy Story,30000000.0,373554033.0,12.451801,1995
1,Joe Johnston,Jumanji,65000000.0,262797249.0,4.043035,1995
2,Howard Deutch,Grumpier Old Men,NaN,NaN,NaN,1995
3,Forest Whitaker,Waiting to Exhale,16000000.0,81452156.0,5.090760,1995
4,Charles Shyer,Father of the Bride Part II,NaN,76578911.0,NaN,1995


### Exportando el dataset para la funcion: recomendacion( titulo )

In [23]:
# Exportamos solo titulo y resumen de la pelicula para aplicar un sistema de recomendacion por similitud de coseno
titulos = dff[["title", "overview"]].sort_values("title", ascending=False)
titulos.reset_index(inplace=True)
titulos.drop(["index"], axis = 1, inplace = True)
titulos.head()

,title,overview
0,ファンタスティポ,Two brothers must take the helm of their famil...
1,…And the Fifth Horseman Is Fear,A Jewish doctor helps a political fugitive dur...
2,’Round Midnight,Inside the Blue Note nightclub one night in 19...
3,‘Rameau’s Nephew’ by Diderot (Thanx to Dennis ...,Described (rather cheekily) by director Michae...
4,هیچ کجا هیچ کس,NaN


In [25]:
titulos.to_csv("Dataset_generados/titulos.csv", index = False)
df = pd.read_csv("Dataset_generados/titulos.csv")
df. head()

,title,overview
0,ファンタスティポ,Two brothers must take the helm of their famil...
1,…And the Fifth Horseman Is Fear,A Jewish doctor helps a political fugitive dur...
2,’Round Midnight,Inside the Blue Note nightclub one night in 19...
3,‘Rameau’s Nephew’ by Diderot (Thanx to Dennis ...,Described (rather cheekily) by director Michae...
4,هیچ کجا هیچ کس,NaN


## Funciones

In [2]:
def peliculas_idioma( Idioma: str ): 
    #Se ingresa un idioma (como están escritos en el dataset, no hay que traducirlos!). Debe devolver la cantidad de películas producidas en ese idioma.
    df = pd.read_csv("Dataset_generados/idiomas.csv")     
    cantidad = df.loc[df.original_language == Idioma]
    cantidad = cantidad.cantidad_peliculas.to_list()
    
    return f"{cantidad[0]} películas fueron estrenadas en idioma {Idioma}"


def peliculas_duracion( Pelicula: str ):
    #Se ingresa una pelicula. Debe devolver la la duracion y el año.
    df = pd.read_csv("Dataset_generados/duracion.csv") 
    titulo = df.loc[df.title == Pelicula]
    duracion = titulo.runtime.to_list()
    año = titulo.release_year.to_list()

    return f"La pelicula {Pelicula} dura {duracion[0]} min y se estreno el año {año[0]}"

def franquicia( Franquicia: str ):
    #Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio
    df = pd.read_csv("Dataset_generados/franquicia.csv")
    franquicia_ingresada = df.loc[df.belongs_to_collection == Franquicia]
    cantidad = franquicia_ingresada.cantidad_peliculas.to_list()
    ganancia_total = franquicia_ingresada.ganancia_total.to_list()
    ganancia_promedio = franquicia_ingresada.promedio_ganancia.to_list()
                    
    return f"La franquicia {Franquicia} posee {cantidad[0]} peliculas, una ganancia total de {ganancia_total[0]} y una ganancia promedio de {ganancia_promedio[0]}"

def peliculas_pais( Pais: str ): 
    #Se ingresa un país (como están escritos en el dataset, no hay que traducirlos!), retornando la cantidad de peliculas producidas en el mismo.
    df = pd.read_csv("Dataset_generados/pais.csv")
    pais_ingresado = df.loc[df.pais == Pais]
    cantidad_peliculas = pais_ingresado.cantidad_peliculas.to_list()

    return f"Se produjeron {cantidad_peliculas[0]} películas en el país {Pais}"

def productoras_exitosas( Productora: str ):
    #Se ingresa la productora, entregandote el revunue total y la cantidad de peliculas que realizo.
    df = pd.read_csv("Dataset_generados/productoras.csv")
    productora_ingresada = df.loc[df.production_companies == Productora]
    revenue = productora_ingresada.revenue.to_list()
    cantidad_peliculas = productora_ingresada.cantidad_peliculas.to_list()
    
    return f"La productora {Productora} ha tenido un revenue total de {revenue[0]} y realizo {cantidad_peliculas[0]} peliculas"

def get_director( nombre_director ):
    #Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno.
    #Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma, en formato lista.
    df1 = pd.read_csv("Dataset_generados/director_1.csv")
    df2 = pd.read_csv("Dataset_generados/director_2.csv")
    director_buscado = df1.loc[df1.director == nombre_director]
    r = director_buscado["return_total"].to_list()

    director_buscado_2 = df2[(df2["crew"]== nombre_director) & (df2["return"].notnull())]
    g = director_buscado_2["title"].to_list()
    a = director_buscado_2["release_year"].to_list()
    rr = director_buscado_2["return"].to_list()
    bd = director_buscado_2["budget"].to_list()
    rv = director_buscado_2["revenue"].to_list()
    return {"director":nombre_director, "retorno_total_director":r, "peliculas":g, "anio":a,
            "retorno_pelicula": rr, "budget_pelicula":bd, "revenue_pelicula":rv}

In [3]:
# ML
def recomendacion(titulo):
    """Ingresa un nombre de película y te recomienda películas similares en una lista"""
    i = pd.read_csv("Dataset_generados/titulos.csv", encoding="utf-8")
    #with open("Dataset_generados/titulos.csv", encoding="utf-8") as file:
    #    i = pd.read_csv(file) #, usecols=['title', 'overview']

    tfidf = TfidfVectorizer(stop_words="english")
    i["overview"] = i["overview"].fillna("")

    tfidf.fit(i["overview"])
    titulo_idx = i[i["title"] == titulo].index[0]
    tfidf_matriz = tfidf.transform([i["overview"][titulo_idx]])

    coseno_sim = linear_kernel(tfidf_matriz, tfidf.transform(i["overview"]))

    simil = list(enumerate(coseno_sim[0]))
    simil = sorted(simil, key=lambda x: x[1], reverse=True)
    simil = simil[1:11]
    movie_index = [i[0] for i in simil]

    lista = i["title"].iloc[movie_index].tolist()[:5]

    return f"Lista recomendada: {lista}"



In [6]:
print(peliculas_idioma("en"))
print(peliculas_duracion("Toy Story"))
print(franquicia("Ace Ventura Collection"))
print(peliculas_pais("United States of America"))
print(productoras_exitosas("20th Century Fox"))
print(get_director("John Lasseter"))
print(recomendacion("The Dark Knight Rises"))

32202 películas fueron estrenadas en idioma en
La pelicula Toy Story dura 81.0 min y se estreno el año 1995
La franquicia Ace Ventura Collection posee 2 peliculas, una ganancia total de 319602929.0 y una ganancia promedio de 159801464.5
Se produjeron 21176 películas en el país United States of America
La productora 20th Century Fox ha tenido un revenue total de 51523297.16666667 y realizo 29 peliculas
{'director': 'John Lasseter', 'retorno_total_director': [21.00525680277778], 'peliculas': ['Toy Story', "A Bug's Life", 'Toy Story 2'], 'anio': [1995, 1998, 1999], 'retorno_pelicula': [12.4518011, 3.027157158333333, 5.526298544444445], 'budget_pelicula': [30000000.0, 120000000.0, 90000000.0], 'revenue_pelicula': [373554033.0, 363258859.0, 497366869.0]}
Lista recomendada: ['The Dark Knight', 'Batman Forever', 'Batman Returns', 'Batman: Under the Red Hood', 'Batman']
